# Preprocessing
This notebook is created during the hackathon on East Africa (March 2024).
It downloads an OSM network based on the combined extent of the various regions that are provided as input folders, containing a file `gis\region.geojson`.

## General imports

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely import geometry, Polygon
from osgeo import gdal, osr
from shutil import copyfile

## RA2CE imports and constants

In [2]:
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum
from ra2ce.network.network_config_data.network_config_data import NetworkConfigData, NetworkSection
from ra2ce.network.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper
from ra2ce.network.exporters.geodataframe_network_exporter import GeoDataFrameNetworkExporter
from ra2ce.network.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter

INPUT_FOLDER = Path('P:\moonshot2-casestudy\SFINCS\models')
INPUT_FLOODMAP_FOLDER = 'floodmaps_wgs84'
OUTPUT_FOLDER = Path('P:\moonshot2-casestudy\RA2CE')
OUTPUT_FLOODMAP_FOLDER = 'floodmaps'

## Determine the polygon of the total extent of the regions
The extent of each region is read from file `region.geojson` and appended in a GeoDataFrame.
From this GeoDataFrame a Polygon is extracted.

In [ ]:
# Loop over region folders and concatenate the extents
_combined_gdf = gpd.GeoDataFrame()
for _region in [f for f in INPUT_FOLDER.iterdir() if f.is_dir()]:
    _extent_file = _region.joinpath('gis', 'region.geojson')
    assert _extent_file.is_file()
    _gdf = gpd.read_file(_extent_file)
    _combined_gdf = pd.concat([_combined_gdf, _gdf.to_crs(4326)], ignore_index=True)

# Extract polygon of the total extent spanning the concatenated regions
_total_extent = _combined_gdf.total_bounds
_polygon = Polygon(geometry.box(*_total_extent))

# Write polygon (not required)
_polygon_file = INPUT_FOLDER.joinpath('polygon.geojson')
_polygon_gdf = gpd.GeoDataFrame(index=[0], geometry=[_polygon], crs='epsg:4326')
_polygon_gdf.to_file(_polygon_file, driver='GeoJSON')

# Write the combined extent to a new file (not required)
_combined_extent_file = INPUT_FOLDER.joinpath('combined_extent.geojson')
_combined_geojson = _combined_gdf.to_json()
with open(_combined_extent_file, 'w') as f:
    f.write(_combined_geojson)



## Create network
The network is downloaded from OSM based on the given polygon.

In [ ]:
# Read network from polygon
_road_type_list = [RoadTypeEnum.MOTORWAY, RoadTypeEnum.MOTORWAY_LINK, RoadTypeEnum.TRUNK, RoadTypeEnum.TRUNK_LINK, RoadTypeEnum.PRIMARY, RoadTypeEnum.PRIMARY_LINK, RoadTypeEnum.SECONDARY, RoadTypeEnum.SECONDARY_LINK, RoadTypeEnum.TERTIARY, RoadTypeEnum.TERTIARY_LINK]
_network_section = NetworkSection(network_type=NetworkTypeEnum.DRIVE, road_types=_road_type_list, save_gpkg=True)
_config_data = NetworkConfigData(network=_network_section, static_path=OUTPUT_FOLDER.joinpath('static'))
_network_wrapper = OsmNetworkWrapper.with_polygon(_config_data, _polygon)


## Export the network to file

In [ ]:
[_graph, _gdf] = _network_wrapper.get_network()

# Export the graph
_exporter = MultiGraphNetworkExporter(basename='base_graph', export_types=['gpkg', 'pickle'])
_exporter.export(export_path=Path(OUTPUT_FOLDER).joinpath('static', 'output_graph'), export_data=_graph)

# Export the network
_exporter = GeoDataFrameNetworkExporter(basename='base_network', export_types=['gpkg', 'pickle'])
_exporter.export(export_path=Path(OUTPUT_FOLDER).joinpath('static', 'output_graph'), export_data=_gdf)


## Prepare floodmaps

In [ ]:
# Reproject and save the floodmaps
_output_folder = OUTPUT_FOLDER.joinpath(OUTPUT_FLOODMAP_FOLDER)
if not _output_folder.exists():
    _output_folder.mkdir(parents=True)

def check_projection(file) -> bool:
    _input_ds = gdal.Open(str(file))
    _source_proj = _input_ds.GetProjection()
    _srs = osr.SpatialReference(wkt=_source_proj)
    if not _srs.IsProjected:
        return False
    return _srs.GetAttrValue('geogcs') == 'WGS 84'

for _region in [f for f in INPUT_FOLDER.iterdir() if f.is_dir()]:
    _models_dir = _region.joinpath(INPUT_FLOODMAP_FOLDER)
    for _floodmap in _models_dir.iterdir():
        _output_file = _output_folder.joinpath(_floodmap.name)
        if not check_projection(_floodmap):
            raise ValueError(f'Floodmap {_floodmap} is not in the right projection')
        copyfile(_floodmap, _output_file)